<a href="https://colab.research.google.com/github/ayushbaghel01/InvestX_asgnmnt/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [559]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
#from datetime import datetime

**NOTE : All the code is same as that is provided in the google meet, The code for finding efficiency of the below model is included in the last code blocks.**

In [560]:
df = pd.read_csv('Train.csv')
print(df.head())
df=df.head(150)

         Date    Stock  liabilities      equity  total_assets  current_assets  \
0  2006-07-31  Stock 1   625.898686  925.086518   1550.985204      675.789905   
1  2006-08-31  Stock 1   407.387467  903.249093   1310.636561     1023.110983   
2  2006-09-29  Stock 1   464.069891  602.668832   1066.738722      709.711673   
3  2006-10-31  Stock 1   855.854937  391.375369   1247.230306      649.979564   
4  2006-11-30  Stock 1   453.954969  956.490566   1410.445534     1115.594551   

   current_liabilities  total_revenue  net_income   dividend  \
0           555.931329     548.488451  251.282553  68.184030   
1          1295.186240     513.446432  207.576940  42.232672   
2           598.719338     503.519871  133.719700  28.835722   
3           733.594339     596.006397  -76.881818   0.000000   
4          1152.274711     965.939247 -309.204301   0.000000   

   shares_outstanding       price  
0             1040831  100.000000  
1             1046649  101.924169  
2             109415

In [561]:
cols = list(df)[2:12]
print(cols)

['liabilities', 'equity', 'total_assets', 'current_assets', 'current_liabilities', 'total_revenue', 'net_income', 'dividend', 'shares_outstanding', 'price']


In [562]:
df_for_training = df[cols].astype(float)

In [563]:
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [564]:
trainX = []
trainY = []

In [565]:
n_future = 1  # Number of days we want to look into the future based on the past days.
n_past = 14 # Number of past days we want to use to predict the future.

In [566]:
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 9])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (136, 14, 10).
trainY shape == (136, 1).


In [567]:
df_for_training_scaled;

In [568]:
trainY;

In [569]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True)) 
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_48 (LSTM)              (None, 14, 64)            19200     
                                                                 
 lstm_49 (LSTM)              (None, 32)                12416     
                                                                 
 dropout_24 (Dropout)        (None, 32)                0         
                                                                 
 dense_24 (Dense)            (None, 1)                 33        
                                                                 
Total params: 31,649
Trainable params: 31,649
Non-trainable params: 0
_________________________________________________________________


In [570]:
history = model.fit(trainX, trainY, epochs=5, batch_size=3, validation_split=0.1, verbose=1)

Epoch 1/5
41/41 [==============================] - 4s 24ms/step - loss: 0.5836 - val_loss: 0.0522
Epoch 2/5
41/41 [==============================] - 1s 16ms/step - loss: 0.1476 - val_loss: 0.0407
Epoch 3/5
41/41 [==============================] - 1s 17ms/step - loss: 0.0672 - val_loss: 0.0371
Epoch 4/5
41/41 [==============================] - 1s 17ms/step - loss: 0.0578 - val_loss: 0.0348
Epoch 5/5
41/41 [==============================] - 1s 16ms/step - loss: 0.0416 - val_loss: 0.0522


In [571]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [572]:
 df2= df.head(150)
train_dates = pd.to_datetime(df2['Date'])

In [573]:
n_past = 1
n_days_for_prediction=50  #let us predict past 15 days

predict_period_dates = pd.date_range(list(train_dates)[-n_past], periods=n_days_for_prediction, freq='M').tolist()
print(predict_period_dates)

[Timestamp('2018-12-31 00:00:00', freq='M'), Timestamp('2019-01-31 00:00:00', freq='M'), Timestamp('2019-02-28 00:00:00', freq='M'), Timestamp('2019-03-31 00:00:00', freq='M'), Timestamp('2019-04-30 00:00:00', freq='M'), Timestamp('2019-05-31 00:00:00', freq='M'), Timestamp('2019-06-30 00:00:00', freq='M'), Timestamp('2019-07-31 00:00:00', freq='M'), Timestamp('2019-08-31 00:00:00', freq='M'), Timestamp('2019-09-30 00:00:00', freq='M'), Timestamp('2019-10-31 00:00:00', freq='M'), Timestamp('2019-11-30 00:00:00', freq='M'), Timestamp('2019-12-31 00:00:00', freq='M'), Timestamp('2020-01-31 00:00:00', freq='M'), Timestamp('2020-02-29 00:00:00', freq='M'), Timestamp('2020-03-31 00:00:00', freq='M'), Timestamp('2020-04-30 00:00:00', freq='M'), Timestamp('2020-05-31 00:00:00', freq='M'), Timestamp('2020-06-30 00:00:00', freq='M'), Timestamp('2020-07-31 00:00:00', freq='M'), Timestamp('2020-08-31 00:00:00', freq='M'), Timestamp('2020-09-30 00:00:00', freq='M'), Timestamp('2020-10-31 00:00:00'

In [574]:
prediction = model.predict(trainX[-n_days_for_prediction:]) 

2/2 [==============================] - 0s 14ms/step


In [575]:
prediction_copies = np.repeat(prediction, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,9]

In [576]:
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i.date())
    
df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), 'price':y_pred_future})
df_forecast['Date']=pd.to_datetime(df_forecast['Date'])

In [577]:
original = df[['Date', 'price']]
original['Date']=pd.to_datetime(original['Date'])


<ipython-input-577-d6bbd2f4d398>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original['Date']=pd.to_datetime(original['Date'])


In [578]:
df_forecast
df_ = pd.read_csv('Train.csv')
df_=df_.head(200)
predicted_price = df_forecast.price

In [579]:
actual_price = df_.price[150:200]

In [580]:
l=[]
for i in range(50):
  l.append(abs(predicted_price[i]-actual_price[150+i])/actual_price[150+i])
deviation_rd=np.array(l)
rms = np.sqrt(np.mean(np.square(deviation_rd)))
efficiency = (1-rms)*100
print(efficiency)
print(l)

82.41016582009371
[0.2697969055175781, 0.23696153643433326, 0.21774041263788324, 0.22783177117135925, 0.23040454588288264, 0.24196399698436852, 0.23774697376683604, 0.24390436477504884, 0.23227216295904873, 0.2194925405761306, 0.20417815995836153, 0.1874537456011981, 0.1718583974186806, 0.16209021370908766, 0.16127457732410033, 0.16081083559854484, 0.15762509645132045, 0.14685516230099196, 0.15013939671252477, 0.1588597511183781, 0.15452705106148826, 0.1563584366293323, 0.1693996638110431, 0.16979362126554734, 0.154296319820768, 0.13305862621074177, 0.1325433232499023, 0.14907979403165583, 0.13938971065417993, 0.12367246944771128, 0.12590995426946477, 0.11952636806328928, 0.12875355095091068, 0.12349127699501895, 0.1305486591507583, 0.1281460434866936, 0.12504705320711995, 0.11392030090672743, 0.12275231627326053, 0.14410578918084158, 0.16274325991465521, 0.15896293063299852, 0.15420140768548876, 0.16209405824316914, 0.16941515583326378, 0.1744516722896381, 0.19147433810631537, 0.20564